In [1]:
import numpy as np 
import cv2
from keras.datasets import mnist


Using TensorFlow backend.


In [0]:
class Convolution:
    def __init__ (self,filters,filter_size):
      self.filters = filters
      self.filter_size = filter_size
      self.conv_filter = np.random.randn(filters,filter_size,filter_size)/(filter_size*filter_size)

    def image_region(self,image):
      height,width = np.shape(image)
      self.image = image
      for i in range(height-self.filter_size+1):
        for j in range(width-self.filter_size+1): 
          img_patch = image[i:(i+self.filter_size),j:(j+self.filter_size)]
          yield img_patch,i,j
    
    def forward_prop(self,image):
      height,width = np.shape(image)
      convo_output = np.zeros((height-self.filter_size+1,width-self.filter_size+1,self.filters))
      for img_patch,i,j in self.image_region(image):
        convo_output[i,j] = np.sum(img_patch*self.conv_filter,axis=(1,2))
      return convo_output
    

    def back_prop(self,dL_dmaxpool,learning_rate):
      dL_dconv = np.zeros(np.shape(self.conv_filter))
      for image_patch,i,j in self.image_region(self.image):
        for k in range(self.filters):
          dL_dconv[k] +=image_patch*dL_dmaxpool[i,j,k]

      self.conv_filter -= learning_rate*dL_dconv
      return dL_dconv





In [0]:
class Maxpool:
  def __init__(self,filter_size):
    self.filter_size = filter_size
  
  def image_region(self,image):
    new_height = np.shape(image)[0]//self.filter_size
    new_width = np.shape(image)[1]//self.filter_size
    self.image = image
    for i in range(new_height):
      for j in range(new_width):
        image_patch = image[(i*self.filter_size):(i*self.filter_size+self.filter_size),(j*self.filter_size):(j*self.filter_size+self.filter_size)]
        yield image_patch,i,j

  def forward_prop(self,image):
    height,width,filters = np.shape(image)
    output1 = np.zeros((height//self.filter_size,width//self.filter_size,filters))
    

    for image_patch,i,j in self.image_region(image):
      # print(i,j)
      output1[i,j] = np.amax(image_patch,axis=(0,1))
    return output1
  
  def back_prop(self,dL_dx):
    dL_dmaxpool = np.zeros(np.shape(self.image))
    for image_patch,i,j in self.image_region(self.image):
      height,width,filters = np.shape(image_patch)
      max_value = np.amax(image_patch,axis=(0,1))

      for t in range(height):
        for u in range(width):
          for v in range(filters):
            if image_patch[t,u,v] == max_value[v]:
              dL_dmaxpool[i*self.filter_size+t,j*self.filter_size+u,v] = dL_dx[i,j,v]

        return dL_dmaxpool

           
      




In [0]:
class Softmax:
  def __init__(self,input_node,softmax_node):
    self.weight = np.random.randn(input_node,softmax_node)/input_node
    self.bias = np.zeros(softmax_node)
  
  def forward_prop(self,image):
    self.original_image_size = np.shape(image)
    flat_image = image.flatten()
    self.flat_image = flat_image
    output_value = np.dot(self.flat_image,self.weight) + self.bias
    self.output_value = output_value
    output_exp = np.exp(self.output_value)
    return output_exp/np.sum(output_exp,axis = 0)

  def back_prop(self,dL_out,learning_rate):
    for i,grad in enumerate(dL_out):
      if grad == 0:
        continue
      transformation_eqn = np.exp(self.output_value)
      S_total = np.sum(transformation_eqn)

      dy_dz = -transformation_eqn[i]*transformation_eqn/(S_total**2)
      dy_dz[i]= transformation_eqn[i]*(S_total - transformation_eqn[i])/(S_total**2)

      dz_dw = self.flat_image
      dz_dbias = 1
      dz_dx = self.weight

      dL_dz = grad*dy_dz
      dL_dw = dz_dw[np.newaxis].T @ dL_dz[np.newaxis]
      dL_dbias = dL_dz*dz_dbias
      dL_dx = dz_dx @ dL_dz
      self.weight -= learning_rate*dL_dw 
      self.bias -=learning_rate*dL_dbias

      return dL_dx.reshape(self.original_image_size)


In [5]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()

train_images = X_train[:1500]
train_labels = y_train[:1500]
test_images = X_test[:1500]
test_labels = y_train[:1500]

conv = Convolution(8,3)
pool =Maxpool(2)
softmax = Softmax(13*13*8,10)

def cnn_forward_prop(image,label):
  out_p =conv.forward_prop((image/255)-0.5)
  out_p =pool.forward_prop(out_p)
  out_p =softmax.forward_prop(out_p)

  cross_entropy_loss = -np.log(out_p[label])
  accurecy = 1 if np.argmax(out_p) == label else 0 
  return out_p,cross_entropy_loss,accurecy


def trainning_cnn(image,label,learn_rate=.005):
  out,loss,accurecy = cnn_forward_prop(image,label)
  gradient = np.zeros(10)
  gradient[label] = -1/out[label]
  grad_back = softmax.back_prop(gradient,learn_rate)
  grad_back = pool.back_prop(grad_back)
  grad_back = conv.back_prop(grad_back,learn_rate)

  return loss,accurecy



11493376/11490434 [==============================] - 2s 0us/step


In [6]:
for epoch in range(4):
  print("Epoch -->%d",(epoch+1))
  shuffle_data = np.random.permutation(len(train_images))
  train_images = train_images[shuffle_data]
  train_labels = train_labels[shuffle_data]


  loss = 0 
  accu = 0
  for i,(im,label) in enumerate(zip(train_images,train_labels)):
    if i%100 == 0:
      print("%d steps out of 100 steps:Avg loss %.2f and accurecy:%d%%",(i+1,loss/100,accu))
      loss = 0 
      accu = 0
    los,acu = trainning_cnn(im,label)
    loss+=los
    accu +=acu


Epoch -->%d 1
%d steps out of 100 steps:Avg loss %.2f and accurecy:%d%% (1, 0.0, 0)
%d steps out of 100 steps:Avg loss %.2f and accurecy:%d%% (101, 2.2185442160168334, 16)
%d steps out of 100 steps:Avg loss %.2f and accurecy:%d%% (201, 2.0361208297774187, 35)
%d steps out of 100 steps:Avg loss %.2f and accurecy:%d%% (301, 1.874230502368805, 44)
%d steps out of 100 steps:Avg loss %.2f and accurecy:%d%% (401, 1.699728795839908, 55)
%d steps out of 100 steps:Avg loss %.2f and accurecy:%d%% (501, 1.6599321132103901, 55)
%d steps out of 100 steps:Avg loss %.2f and accurecy:%d%% (601, 1.4813138573198588, 68)
%d steps out of 100 steps:Avg loss %.2f and accurecy:%d%% (701, 1.3683083175329347, 68)
%d steps out of 100 steps:Avg loss %.2f and accurecy:%d%% (801, 1.229172515060484, 78)
%d steps out of 100 steps:Avg loss %.2f and accurecy:%d%% (901, 1.1348529504207152, 78)
%d steps out of 100 steps:Avg loss %.2f and accurecy:%d%% (1001, 1.259355425930504, 68)
%d steps out of 100 steps:Avg loss %.2f

In [8]:
#Test things
test_loss = 0
test_accu = 0

for im,labels in zip(test_images,test_labels):
  out_value,los,acu = cnn_forward_prop(im,labels)
  test_loss +=los
  test_accu +=acu

test_count = len(test_images)
print("Test Loss %.2f",test_loss/test_count)
print("Test Accurecy %.2f",test_accu/test_count)

Test Loss %.2f 4.0718563840058675
Test Accurecy %.2f 0.09933333333333333
